# OBS data and metadata

## Set up local paths and functions to read data/metadata either locally or online

In [ ]:
from read_data_metadata import read_data, read_metadata, DATA_PATH

help(read_data)
help(read_metadata)
print(f"{DATA_PATH=}")

## Response stages

In [ ]:
"""
Download metadata (inventory file)
"""
# Declare parameters for the data/metadata to look at
nslc, st, et, server = "YV.RR29.*.B*", "2013-02-05T00", "2013-02-13T00", "RESIF"

# Read metadata
inv = read_metadata(server, nslc, st, et)

# select the Z channel's response
# Select returns an inventory, networks are inventory items, stations are network items, channels are station items
# Strangely, obspy's documentation does not mention "response" as an attribute.
resp = inv.select(channel='*Z')[0][0][0].response
stages = resp.response_stages
n_stages = len(stages)
output_sampling_rate = 62.5

sensor_stage = 1
preamp_stage = -1 # No preamp stage
ad_stage = 2

# Calculate total decimation and input sample rate
total_decim=1
for stage in stages[ad_stage: ]:
    total_decim *= stage.decimation_factor
ad_sampling_rate = total_decim * output_sampling_rate

# Plot individual stages
for i in range(1, n_stages+1):
    stage = stages[i-1]
    if i == sensor_stage:
        min_freq, sr, descr = 0.001, output_sampling_rate, 'sensor'
    elif i == preamp_stage:
        min_freq, sr, descr = 1, output_sampling_rate, 'preamplifier'
    elif i == ad_stage:
        min_freq, sr, descr = 1, ad_sampling_rate, 'A/D'
    else:
        if not i > ad_stage:
            raise ValueError('there must be a hole in the declared stage numbers: '
                             f'{sensor_stage=}, {preamp_stage=}, {ad_stage=}')
        min_freq, sr, descr = 1, sr/stage.decimation_factor, f'FIR stage {i - ad_stage}'
    if stage.description is not None:
        descr = stage.description
    resp.plot(min_freq, start_stage=i, end_stage=i, sampling_rate=sr, label=descr)

# Plot all FIR stages
resp.plot(1., start_stage=ad_stage+1, end_stage=n_stages, sampling_rate=ad_sampling_rate, label="FIRs")  # => 4000 sps

# Plot entire response
resp.plot(0.001, label="BBOBS")


## PPSDs

In [ ]:
"""
Plot probabilistic power spectral densities for a station
"""
from obspy import read
from obspy.signal.spectral_estimation import PPSD
from matplotlib import pyplot as plt


# Read data and metadata
stream = read_data(server, nslc, st, et)   # data
inv = read_metadata(server, nslc, st, et)  # metadata

print(stream)

def make_plot_ppsd(stream, inv, ch, kwargs={}):
    """Create and plot PPSD object"""
    tr = stream.select(channel=ch)[0]
    ppsd = PPSD(tr.stats, inv, **kwargs)  
    ppsd.add(stream)

    # Plot PPSD and spectrogram
    ppsd.plot()
    ppsd.plot_spectrogram()

    return ppsd


# Create and feed PPSD object for "Z" and "H" channels
ppsdz = make_plot_ppsd(stream, inv, "*Z", kwargs={})
ppsdh = make_plot_ppsd(stream, inv, "*H", kwargs={"special_handling": "hydrophone",
                                                  "db_bins": (-50, 60, 1.)})


## Mean Power Spectral Densities

In [ ]:
# Read several stations, just Z channel and one day
nslc, st, et, server = "YV.RR2*.*.*Z", "2013-02-05T00", "2013-02-06T00", "RESIF"

# Read data and metadata
stream = read_data(server, nslc, st, et)
inv = read_metadata(server, nslc, st, et)

print(stream)
inv.plot(projection='local')  # "local" was much slower than "global" and "ortho" the first time

f, ax = plt.subplots()
for tr in stream:
    ppsd = PPSD(tr.stats, inv)
    ppsd.add(tr)
    # periods, values = ppsd.get_percentile(50.)
    periods, values = ppsd.get_mean()
    ax.semilogx(periods, values, label=tr.stats.station)
ax.set_xlabel('Period (s)')
ax.set_ylabel('dB ref m/s^2')
ax.set_title(f'Mean {nslc} PSDs')
ax.legend()
plt.show()
    

## Creating a (basic) inventory file
This lesson shows you the basic components of an inventory file, focussing on the response.  Creating and validating a real instrument response requires
a lot more lines or, better yet, a specific tool like Yasmine or obsinfo.
### Create a response
We will make a response consisting of a seismometer, a simple gain stage and a simplified datalogger

In [ ]:
"""
Create a seismometer channel response.  The seismometer comes from the Trillium Compact OBS manual, the FIR from the TI1284 section 1
(this example ignores sections 2 and 3)
"""
from obspy.core.inventory.response import (FIRResponseStage, PolesZerosResponseStage, CoefficientsTypeResponseStage,
                                           Response, InstrumentSensitivity)

# Note: if one decimation variable is specified, all 5 must be specified!
stages = [PolesZerosResponseStage(1, 754.3, 1.0, "m/s", "V", "LAPLACE (RADIANS/SECOND)", 1.0,
                                  [0., 0., -392., -1960., -1490 + 1j*1740, -1490 - 1j*1740],
                                  [-0.03691 + 1j*0.03702, -0.03691 - 1j*0.03702, -343, -370 + 1j*467, -370 - 1j*467,
                                   -836 + 1j*1522,  -836 - 1j*1522, -4900 + 1j*4700, -4900 - 1j*4700, -6900, -15000],
                                  4.34493e17, description="seismometer") ,
            PolesZerosResponseStage(2, 2., 1.0, "V", "V", "LAPLACE (RADIANS/SECOND)", 1.0, [], [], description="2x gain preampli"),
            CoefficientsTypeResponseStage(3, 14030000, 1.0, "V", "count", "DIGITAL", numerator=[], denominator=[],
                                        decimation_input_sample_rate=1000., decimation_factor=1, decimation_delay=0, decimation_correction=0, decimation_offset=0,
                                        description="A/D"),
            FIRResponseStage(4, 1., 1.0, "count", "count", "ODD", [3, 0, -25, 0, 150, 256],
                           decimation_input_sample_rate=1000., decimation_factor=2, decimation_delay=5, decimation_correction=5, decimation_offset=0,
                           description="FIR filter")
]
response = Response(response_stages=stages, instrument_sensitivity=InstrumentSensitivity(1., 1., "m/s", "count"))
response.recalculate_overall_sensitivity(1.0)
print(response)
response.plot(min_freq=0.001, label="my fake seismometer channel")

### Create an inventory containing the response

In [ ]:
"""
Create and save an inventory, using (one) existing response
"""
from obspy.core.inventory import Channel, Station, Network, Inventory

lat, lon, elev, depth = 49.18, -2.15, -10., 0.  # What is the lat, lon precision in meters?
channels = [Channel(name, '00', lat, lon, elev, depth, response=response)
            for name in ('CHZ', 'CH1', 'CH2', 'CDG')]
stations = [Station('JERB', lat, lon, elev, channels=channels)]
networks = [Network('XX', stations=stations)]
inventory = Inventory(networks=networks)
print(inventory)
inventory.plot(projection='local')
inventory.plot_response(0.001)

# This doesn't work!!!
inventory.write('my_inventory.xml', 'STATIONXML')

## Exercises

- Why did I call the channels "C*" instead of "B*"?
- Use obspy to read in example.station.xml and write to all the different available formats
(``CSS``, ``KML``, ``SACPZ``, ``SHAPEFILE``, ``STATIONTXT``).
Compare the results to see what "levels" are available and for what
purpose you might use each one.
- Why doesn't ``inventory.write()`` work in the last example?